# Kerasのサンプル

バージョン確認

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.6'

MNISTデータのロード

In [2]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

入力をフラットな配列に変換し、学習データは10,000個、テストデータは1000個だけ利用します。また、元データでは濃淡が0から255までで表現されており、最大値が大きすぎるので、特徴の値の最大値を1としておきます。

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows * img_cols)[0:10000]
X_test = X_test.reshape(X_test.shape[0], img_rows * img_cols)[0:1000]
input_shape = (img_rows, img_cols)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

出力をone-hotに変換します。

In [4]:
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train)[0:10000]
Y_test = np_utils.to_categorical(y_test)[0:1000]

## 多層パーセプトロン (MLP)

準備

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

まず、中間層1層のモデルを設定します。ユニット数は入力の次元数を考えて、適切に設定してください。活性化関数は、パラメータ[activation](https://keras.io/ja/activations/)で指定します。

In [6]:
input_shape = X_train[0].shape
n_hidden = 200
n_out = len(Y_train[0])  # 10

model = Sequential()
model.add(Dense(n_hidden, input_shape=input_shape, activation='sigmoid'))
model.add(Dense(n_out, activation='softmax'))

損失関数 ([loss](https://keras.io/ja/losses/))、最適化アルゴリズム([optimizer](https://keras.io/ja/optimizers/))、評価基準([metrics](https://keras.io/ja/metrics/))を指定して、モデルをcompileします。

In [7]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])

[fit](https://keras.io/ja/models/sequential/#fit)メソッドで学習します。繰り返し回数はパラメータepocksで、バッチサイズはパラメータbatch\_sizeで指定します。

In [8]:
model.fit(X_train, Y_train, epochs=10, batch_size=200)

Epoch 1/10
10000/10000 [==============================] - 0s - loss: 1.9751 - acc: 0.4757     
Epoch 2/10
10000/10000 [==============================] - 0s - loss: 1.4174 - acc: 0.7287     
Epoch 3/10
10000/10000 [==============================] - 0s - loss: 1.0610 - acc: 0.7948     
Epoch 4/10
10000/10000 [==============================] - 0s - loss: 0.8530 - acc: 0.8282     
Epoch 5/10
10000/10000 [==============================] - 0s - loss: 0.7253 - acc: 0.8498     
Epoch 6/10
10000/10000 [==============================] - 0s - loss: 0.6400 - acc: 0.8596     
Epoch 7/10
10000/10000 [==============================] - 0s - loss: 0.5811 - acc: 0.8683     
Epoch 8/10
10000/10000 [==============================] - 0s - loss: 0.5368 - acc: 0.8738     
Epoch 9/10
10000/10000 [==============================] - 0s - loss: 0.5027 - acc: 0.8791     
Epoch 10/10
10000/10000 [==============================] - 0s - loss: 0.4757 - acc: 0.8838     


[evaluate](https://keras.io/ja/models/sequential/#evaluate)メソッドで評価します。

In [9]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.532082397461
Test accuracy: 0.861


### 中間層の層数を増やす

中間層の層数を増やしてみます。その際、活性化関数sigmoid, tanh, reluのそれぞれの違いを確認します。

In [10]:
model = Sequential()
af = 'sigmoid'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_out, activation='softmax'))

In [11]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
10000/10000 [==============================] - 0s - loss: 2.3161 - acc: 0.1019     
Epoch 2/10
10000/10000 [==============================] - 0s - loss: 2.3098 - acc: 0.1059     
Epoch 3/10
10000/10000 [==============================] - 0s - loss: 2.3109 - acc: 0.1024     
Epoch 4/10
10000/10000 [==============================] - 0s - loss: 2.3112 - acc: 0.1052     
Epoch 5/10
10000/10000 [==============================] - 0s - loss: 2.3099 - acc: 0.1028     
Epoch 6/10
10000/10000 [==============================] - 0s - loss: 2.3099 - acc: 0.1066     
Epoch 7/10
10000/10000 [==============================] - 0s - loss: 2.3098 - acc: 0.0993     
Epoch 8/10
10000/10000 [==============================] - 0s - loss: 2.3093 - acc: 0.1007     
Epoch 9/10
10000/10000 [==============================] - 0s - loss: 2.3074 - acc: 0.1052     
Epoch 10/10
10000/10000 [==============================] - 0s - loss: 2.3084 - acc: 0.1013     
Test loss: 2.30435621071
Test accuracy: 0.116


In [12]:
model = Sequential()
af = 'tanh'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_out, activation='softmax'))

In [13]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
10000/10000 [==============================] - 0s - loss: 0.7945 - acc: 0.7884     
Epoch 2/10
10000/10000 [==============================] - 0s - loss: 0.3765 - acc: 0.8968     
Epoch 3/10
10000/10000 [==============================] - 0s - loss: 0.3097 - acc: 0.9126     
Epoch 4/10
10000/10000 [==============================] - 0s - loss: 0.2751 - acc: 0.9215     
Epoch 5/10
10000/10000 [==============================] - 0s - loss: 0.2552 - acc: 0.9257     
Epoch 6/10
10000/10000 [==============================] - 0s - loss: 0.2349 - acc: 0.9317     
Epoch 7/10
10000/10000 [==============================] - 0s - loss: 0.2116 - acc: 0.9379     
Epoch 8/10
10000/10000 [==============================] - 0s - loss: 0.1967 - acc: 0.9428     
Epoch 9/10
10000/10000 [==============================] - 0s - loss: 0.1802 - acc: 0.9474     
Epoch 10/10
10000/10000 [==============================] - 0s - loss: 0.1732 - acc: 0.9482     
Test loss: 0.288588508368
Test accuracy: 0.91


In [14]:
model = Sequential()
af = 'relu'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_hidden, activation=af))
model.add(Dense(n_out, activation='softmax'))

In [15]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=10, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
10000/10000 [==============================] - 0s - loss: 1.7704 - acc: 0.5010     
Epoch 2/10
10000/10000 [==============================] - 0s - loss: 0.6877 - acc: 0.7868     
Epoch 3/10
10000/10000 [==============================] - 0s - loss: 0.3903 - acc: 0.8854     
Epoch 4/10
10000/10000 [==============================] - 0s - loss: 0.3127 - acc: 0.9067     
Epoch 5/10
10000/10000 [==============================] - 0s - loss: 0.2559 - acc: 0.9237     
Epoch 6/10
10000/10000 [==============================] - 0s - loss: 0.2214 - acc: 0.9327     
Epoch 7/10
10000/10000 [==============================] - 0s - loss: 0.1796 - acc: 0.9485     
Epoch 8/10
10000/10000 [==============================] - 0s - loss: 0.1746 - acc: 0.9474     
Epoch 9/10
10000/10000 [==============================] - 0s - loss: 0.1405 - acc: 0.9605     
Epoch 10/10
10000/10000 [==============================] - 0s - loss: 0.1416 - acc: 0.9578     
Test loss: 0.193576118708
Test accuracy: 0.94


dropoutを入れてみます。学習データとテストデータの正解率の差に注目してください。

In [16]:
from keras.layers.core import Dropout

model = Sequential()
af = 'relu'
model.add(Dense(n_hidden, input_shape=input_shape, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_hidden, activation=af))
model.add(Dropout(0.5))
model.add(Dense(n_out, activation='softmax'))

In [17]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = SGD(lr=0.1),
              metrics = ['accuracy'])
model.fit(X_train, Y_train, epochs=20, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/20
10000/10000 [==============================] - 1s - loss: 2.3216 - acc: 0.1174     
Epoch 2/20
10000/10000 [==============================] - 0s - loss: 2.2211 - acc: 0.1737     
Epoch 3/20
10000/10000 [==============================] - 0s - loss: 1.9988 - acc: 0.2868     
Epoch 4/20
10000/10000 [==============================] - 1s - loss: 1.7006 - acc: 0.4221     
Epoch 5/20
10000/10000 [==============================] - 1s - loss: 1.4405 - acc: 0.4921     
Epoch 6/20
10000/10000 [==============================] - 1s - loss: 1.2276 - acc: 0.5561     
Epoch 7/20
10000/10000 [==============================] - 0s - loss: 1.0981 - acc: 0.5921     
Epoch 8/20
10000/10000 [==============================] - 0s - loss: 1.0066 - acc: 0.6358     
Epoch 9/20
10000/10000 [==============================] - 0s - loss: 0.9306 - acc: 0.6763     
Epoch 10/20
10000/10000 [==============================] - 1s - loss: 0.8546 - acc: 0.7089     
Epoch 11/20
10000/10000 [========================